### Will a Customer Accept the Coupon?

**Context**

Imagine driving through town and a coupon is delivered to your cell phone for a restaraunt near where you are driving. Would you accept that coupon and take a short detour to the restaraunt? Would you accept the coupon but use it on a sunbsequent trip? Would you ignore the coupon entirely? What if the coupon was for a bar instead of a restaraunt? What about a coffee house? Would you accept a bar coupon with a minor passenger in the car? What about if it was just you and your partner in the car? Would weather impact the rate of acceptance? What about the time of day?

Obviously, proximity to the business is a factor on whether the coupon is delivered to the driver or not, but what are the factors that determine whether a driver accepts the coupon once it is delivered to them? How would you determine whether a driver is likely to accept a coupon?

**Overview**

The goal of this project is to use what you know about visualizations and probability distributions to distinguish between customers who accepted a driving coupon versus those that did not.

**Data**

This data comes to us from the UCI Machine Learning repository and was collected via a survey on Amazon Mechanical Turk. The survey describes different driving scenarios including the destination, current time, weather, passenger, etc., and then ask the person whether he will accept the coupon if he is the driver. Answers that the user will drive there ‘right away’ or ‘later before the coupon expires’ are labeled as ‘Y = 1’ and answers ‘no, I do not want the coupon’ are labeled as ‘Y = 0’.  There are five different types of coupons -- less expensive restaurants (under \\$20), coffee houses, carry out & take away, bar, and more expensive restaurants (\\$20 - \\$50). 

**Deliverables**

Your final product should be a brief report that highlights the differences between customers who did and did not accept the coupons.  To explore the data you will utilize your knowledge of plotting, statistical summaries, and visualization using Python. You will publish your findings in a public facing github repository as your first portfolio piece. 





### Data Description
Keep in mind that these values mentioned below are average values.

The attributes of this data set include:
1. User attributes
    -  Gender: male, female
    -  Age: below 21, 21 to 25, 26 to 30, etc.
    -  Marital Status: single, married partner, unmarried partner, or widowed
    -  Number of children: 0, 1, or more than 1
    -  Education: high school, bachelors degree, associates degree, or graduate degree
    -  Occupation: architecture & engineering, business & financial, etc.
    -  Annual income: less than \\$12500, \\$12500 - \\$24999, \\$25000 - \\$37499, etc.
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she buys takeaway food: 0, less than 1, 1 to 3, 4 to 8 or greater
    than 8
    -  Number of times that he/she goes to a coffee house: 0, less than 1, 1 to 3, 4 to 8 or
    greater than 8
    -  Number of times that he/she eats at a restaurant with average expense less than \\$20 per
    person: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    

2. Contextual attributes
    - Driving destination: home, work, or no urgent destination
    - Location of user, coupon and destination: we provide a map to show the geographical
    location of the user, destination, and the venue, and we mark the distance between each
    two places with time of driving. The user can see whether the venue is in the same
    direction as the destination.
    - Weather: sunny, rainy, or snowy
    - Temperature: 30F, 55F, or 80F
    - Time: 10AM, 2PM, or 6PM
    - Passenger: alone, partner, kid(s), or friend(s)


3. Coupon attributes
    - time before it expires: 2 hours or one day

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px


### Problems

Use the prompts below to get started with your data analysis.  

#### 1. Read in the `coupons.csv` file.




In [ ]:
# Load data from source
data = pd.read_csv('data/coupons.csv')

In [ ]:
# Review first 10 rows
data.head(10)

<b>The following observations are gathered from this initial step:</b>
- The columns between "maritalStatus" and "CoffeeHouse" are hidden. We'll explore the values of those in subsequent steps.
- At least on the first 10 records, the "CarryAway" column contains "NaN" (not a number) values
- Values on the "age" column on the first 10 records are set to "21". We'll explore further
- There is a typo on the name of the column "passanger". It shoould be "passenger".

#### 2. Investigate the dataset for missing or problematic data.


In [ ]:
# Get overview of the data structure 
data.info()

<b>From this initial overview, the following observations are gathered:</b>
- There are a total of 12,684 entries and 26 columns (including the index)
- The following columns contain null values (based on the count of "non-null" values for each column):
    - car
    - Bar
    - CoffeeHouse
    - CarryAway
    - RestaurantLessThan20
    - Restaurant20To50
- The column "car" is of type "object". We'll explore its content on the next step since the information on the data showed from the head() method on step 1 is truncated. 
- The following columns are of type "int64":
    - has_children
    - toCoupon_GEQ5min	
    - toCoupon_GEQ15min	
    - toCoupon_GEQ25min	
    - direction_same
    - direction_opp
- Inconsistent naming convention across column names . Although not critical for the data analysis, we may benefit from relabling column names to increase readibility 
- Although describe in the data description provided, the name of the column "Y" is ambigous. We may consider rename it  


In [ ]:
# Explore truncated data
data[['has_children', 'education', 'occupation', 'income', 'car', 'Bar']].head(10)

<b>The following obeservations are gathered from the previous process: </b>
- At least on the first 10 entries, the "car" column contains "NaN" values. We'll continue exploring its content on the next step


We'll explore the columns that we suspect might have problematic values (these include columns with "null" values):

In [ ]:
# Count unique values on the "car" column
data['car'].value_counts()

<b>The following observations are gathered from the previous step:</b>
- There are a total of 5 unique values on this column.
- The values are ambiguous:
    - "Scooter and motorcycle" seems to belong to entries where the person uses a motorcycle (or scooter) instead of a car. Therefore, technically they don't own/drive a car.
    - From the entries with value "Mazda5" (a car model), we can assume that the person own/drive a car. 
    - We may also assume thet entries with the "do not drive" value belong to people that don't own/drive a car
    - From the entries with value "crossover" (type of car), we can assume that the person own/drive a car.
    - From the entries with value "Car that is too old to install Onstar" (description of a car), we can assume that the person own/drive a car. 
- The rest of the data is on the form of "null" values, as we discovered with the info() method above.

In [ ]:
# Count unique values on the "CarryAway" column
data['CarryAway'].value_counts()

In [ ]:
# Count unique values on the "Bar" column
data['Bar'].value_counts()

In [ ]:
# Count unique values on the "CoffeeHouse" column
data['CoffeeHouse'].value_counts()

In [ ]:
# Count unique values on the "CoffeeHouse" column
data['RestaurantLessThan20'].value_counts()

In [ ]:
# Count unique values on the "Restaurant20To50" column
data['Restaurant20To50'].value_counts()

<b>The following information is gathered through the value_counts() method from the previous steps:</b>
- The values of the columns "CarreyAway", "Bar", "CoffeeHouse", "RestaurantLessThan20", "Restaurant20To50" seem descriptive enough
- Those values are consistent across those columns
- Although not a high priority, we may consider ralabling the value "gt8" to increase readibility. 

In [ ]:
# Explore values on the "age" column
data['age'].value_counts()

Based on the data returned from the value counts on the "age" columns, we can assume those values refer to age groups.

#### 3. Decide what to do about your missing data -- drop, replace, other...

Based the observations gathered thus far, we will proceed to clean the data as follow:
1. Correct name of "passanger" column to "passenger"
2. Rename columns using a standard naming convention. We'll use snake_case
3. Create a new column named "motorcycle" using the entries with values "Scooter or motorcycle" from the "car" column
4. Convert the car column to True/False (boolean) values based on whether the person drives (or NOT) a car. The reasoning behind this is that additional information on this column (i.e."Mazda5", "crossover", etc.) does not seem to be relevant for the analysis  
5. Convert "NaN" values to string values on the following columns:
    - bar : 'never'
    - coffee_house : 'never'
    - carry_away : 'never' 
    - restaurant_less_than_20 : 'never'
    - restaurant_20_to_50 : 'never'
6. Rename "Y" column to "accept_coupon"
7. Rename values of "age" column to better describe age groups


In [ ]:
# 1. Rename columns using a standard naming convention. We'll use snake_case
data = data.rename({
    'passanger' : 'passenger'
 }, axis="columns")

# 2. Rename columns using a standard naming convention. We'll use snake_case
data = data.rename({
    'maritalStatus' : 'marital_status',
    'Bar' : 'bar',
    'CoffeeHouse' : 'coffee_house',
    'CarryAway' : 'carry_away',
    'RestaurantLessThan20' : 'restaurant_less_than_20',
    'Restaurant20To50' : 'restaurant_20_to_50',
    'toCoupon_GEQ5min' : 'to_coupon_geq_5min',
    'toCoupon_GEQ15min' : 'to_coupon_geq_15min',
    'toCoupon_GEQ25min' : 'to_coupon_geq_25min'
 }, axis="columns")


In [ ]:
# 3. Create a new column named "rides_motorcycle" using the entries with values "scooter or motorcycle" from the "car" column
data['rides_motorcycle'] = data['car'].apply(lambda x: x == 'Scooter and motorcycle')

# Verify
data['rides_motorcycle'].value_counts()


In [ ]:
# 3. We'll create a "drives_car" column to hold the result of the data conversion for the entries that contain "car related" values. 
    # Scooter and motorcycle                      22
    # Mazda5                                      22
    # do not drive                                22
    # crossover                                   21
    # Car that is too old to install Onstar :D    21

data['drives_car'] = data['car'].apply(lambda x: (x == 'Mazda5') | (x == 'crossover') | (x == 'Car that is too old to install Onstar :D'))
data['drives_car'].value_counts()

In [ ]:
# verify
data.info()

In [ ]:
# 4. Convert "NaN" values to numeric/string values on the following columns:
#   - drives_car : False
#   - bar : 'never'
#   - coffee_house : 'never'
#   - carry_away : 'never 
#   - restaurant_less_than_20 : 'never'
#   - restaurant_20_to_50 : 'never'

data['drives_car'] = data['drives_car'].fillna(False)
data['bar'] = data['bar'].fillna('never')
data['coffee_house'] = data['coffee_house'].fillna('never')
data['carry_away'] = data['carry_away'].fillna('never')
data['restaurant_less_than_20'] = data['restaurant_less_than_20'].fillna('never')
data['restaurant_20_to_50'] = data['restaurant_20_to_50'].fillna('never')

#verify that there are no null values
data.info()

In [ ]:
# 5. Rename "Y" column to "accept_coupon"
data = data.rename({'Y':'accept_coupon'}, axis='columns')

#verify
data.head()

In [ ]:
# 6. Rename values of "age" column to better describe age groups 
def convertToAgeGroup(x):
    match x: 
        case 'below21':
            convertedValue = 'under 21'
        case '21':
            convertedValue = '21 to 25'
        case '26':
            convertedValue = '26 to 30'
        case '31':
            convertedValue = '31 to 35'
        case '31':
            convertedValue = '31 to 35'
        case '36':
            convertedValue = '36 to 40'
        case '41':
            convertedValue = '41 to 45'
        case '46':
            convertedValue = '46 to 50'
        case '50plus':
            convertedValue = 'Over 50'
        case _:
            convertedValue = x            
    return convertedValue

data['age'] = data['age'].apply(lambda x: convertToAgeGroup(x))

# verify
data['age'].value_counts()

#### 4. What proportion of the total observations chose to accept the coupon? 



In [ ]:
# Generate pie chart
fig = px.pie(data, names='accept_coupon', title="Coupon Acceptance Rate: All Coupons")
fig.show()

Response: From the pie chart above we can see that 56.8% of the coupons were accepted

#### 5. Use a bar plot to visualize the `coupon` column.

In [ ]:
# Generate bar chart
fig = px.bar(data['coupon'].value_counts(), title='Coupon count aggregated by coupon type')
fig.show()


6. Use a histogram to visualize the temperature column.

In [ ]:
# Generate histogram
px.histogram(data, x='temperature', title="Coupon count aggregated by temperature when the coupon is accepted or not")

### **Investigating the Bar Coupons**

Now, we will lead you through an exploration of just the bar related coupons.  

#### 1. Create a new `DataFrame` that contains just the bar coupons.


In [ ]:
# Query entries that match condition "coupon == 'Bar'"  
bar_coupons = data[data["coupon"] == 'Bar']

# Verify
bar_coupons.head()

#### 2. What proportion of bar coupons were accepted?


In [ ]:
# Generate pie chart
fig_2 = px.pie(bar_coupons, names='accept_coupon', title="'Bar' Coupon Acceptance Rate")
fig_2.show()

Response: From the previous bar chart we can see that 41% of the bar coupons were accepted

#### 3. Compare the acceptance rate between those who went to a bar 3 or fewer times a month to those who went more.


In [ ]:
# Set list of keys for "three times or less" visits per month
three_or_less = ['never', 'less1', '1~3']

# Set group 3a: those who went to a bar 3 or fewer times in a month
group_3a = bar_coupons.query('bar in @three_or_less')

# Generate visualization for group 3a
fig_3a=px.pie(group_3a, names='accept_coupon', title="'Bar' Coupon Acceptance Rate: Three or less visits in a month to the bar")
fig_3a.show()


From the pie chart above we can see that 37.1% of the bar coupons were accepted by people that visit a bar three or less times in a month.

In [ ]:
# Set group 3b: those who went to a bar more than 3 times in a month
group_3b = bar_coupons.query('bar not in @three_or_less')

# Generate visualization for group 3b
fig_3b=px.pie(group_3b, names='accept_coupon', title="'Bar' Coupon Acceptance Rate: More than three visits a month to the bar")
fig_3b.show()

From the pie chart above we can see that 23.1% of the bar coupons were accepted by people that visit a bar more than three times in a month.

#### 4. Compare the acceptance rate between drivers who go to a bar more than once a month and are over the age of 25 to the all others.  Is there a difference?


In [ ]:
# Create list with values equivalent to 'more than once a month'  
bc_more_than_once = ['1~3', '4~8', 'gt8']

# Create list with values equivalent to 25 years and younger. This will be used as the exclusion condition (not in) for the age feature  
bc_25_years_old_and_under = ['under 21', '21 to 25']

# Create dataframe with filtered data
bc_more_than_once_over_25_driver = bar_coupons.query('drives_car == True & bar in @bc_more_than_once & age not in @bc_25_years_old_and_under')

# Explore results
bc_more_than_once_over_25_driver.describe()


From the previous table we can see that query did not find any matches. Therefore it is not possible to calculate the acceptance rate using those query conditions. However, we're going to also query the data using the column we added with the name "rides_motorcycle" to determine if there are any matches, although technically a motorcyle is not a "drivable" car.

In [ ]:
# Create dataframe with the results of the query "rides_motorcyle" instead of "drives_car". 
bc_more_than_once_over_25_driver = bar_coupons.query('rides_motorcycle == True & bar in @bc_more_than_once & age not in @bc_25_years_old_and_under')

# Explore results
bc_more_than_once_over_25_driver.describe()

The query did not find any matches for the condition "rides_motorcycle == True" either. To be completely sure there was not an issue with the data conversion during the data cleaning process, we're going to perform the original query, but this time using the "car" column

In [ ]:
# Create dataframe with the results of the query using the original "car" column
bc_more_than_once_over_25_driver = bar_coupons.query('car != "do not drive" & bar in @bc_more_than_once & age not in @bc_25_years_old_and_under')

# Explore results (info)
bc_more_than_once_over_25_driver.info()

We were able to obtain 418 entries that do NOT match the value "do not drive". However, we can also verify that all those entries have a NULL "car" value. During our initial data eploration, we assumed that if the value of the "car" column is NULL, the person does not drive. Based on that assumption, we set the values of both the "drives_car" and "rides_motorcycle" columns to FALSE. However, there can be another interpretation of the car value being NULL: the person DOES drive a car (or rides a motorcycle), but the person did not provide a description of such vehicle. 

<b><i>Note: Although another interpretation for the NULL value of "car" could be that the person is not the driver. However, we can dismiss such assumption based on the following statement from the dataset description: "The survey describes different driving scenarios including the destination, current time, weather, passenger, etc., and then ask the person whether he will accept the coupon if he is the driver". This can be also confirmed by exploring the values on the passanger's column for these</i></b> 



In [ ]:
# Explore values of the "passenger" column   
bc_more_than_once_over_25_driver['passenger'].value_counts()

Having confirmed that the passenger column contains relevant data about passengers, we can assume that the person is a driver but that no information is available about the vehicle. For this reason, we're going to continue of our analysis using the original values on the "car" column to determine the coupon acceptance rate. 

In [ ]:
# Generate pie chart
fig = px.pie(bc_more_than_once_over_25_driver, names="accept_coupon", title="'Bar' Coupon Acceptance Rate: Over 25 year old, driver (includes unknown vehicle), visits bar more than once a month")
fig.show()

To calculate the acceptance rate of the rest of the population, we're going to create a copy of the "bar_coupon" dataframe and remove the entries that match the condition "Over 25 year old, driver (including null values) and that visit the bar more than once.

In [ ]:
# Make copy of the "bar_coupons" dataframe
bc_everyone_else = bar_coupons.copy()

# Drop entries what match the condition "Over 25 year old, driver (including null values) and that visit the bar more than once"
bc_everyone_else.drop(bc_more_than_once_over_25_driver.index, inplace=True)

# Plot acceptance rate using pie chart
fig = px.pie(bc_everyone_else, names="accept_coupon", title="'Bar' Coupon Acceptance Rate: Everyone other than 'Over 25 year old, driver (including 'unknown' vehicle) and that visits a bar more than once a month'")
fig.show()

<b>Summary of coupon acceptance rates for groups in question 4:</b>
- Drivers who go to a bar more than once a month and are over the age of 25: 
    - Acceptance rate: 69.4%
- All others: 
    - Acceptance rate: 33.6%

#### 5. Use the same process to compare the acceptance rate between drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry. 


In [ ]:
# Explore unique values for "passengers" 
bar_coupons['passenger'].value_counts()

In [ ]:
# Explore unique values for "occupations" 
bar_coupons['occupation'].value_counts()

In [ ]:
# Group 5: "drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry"
group_5a = bar_coupons.query('car != "do not drive" & bar in @bc_more_than_once & passenger != "Kid(s)" & occupation != "Farming Fishing & Forestry"')

# Generate pie chart
fig_5a = px.pie(group_5a, names='accept_coupon', title="'Bar' Coupon Acceptance Rate: Drivers that go to bars more than once a month, without kid passengers, and had occupations other than farming, fishing, or forestry")
fig_5a.show()

<i>NOTE: It is not clear on the question statement what this group should be compared to. Therefore, we'll continue our analysis with the assumption that the comparison should be done against "All Others" (as in question 4).</i>

In [ ]:
# Group 5b: All others (than "drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry")
group_5b = bar_coupons.copy()
group_5b.drop(group_5a.index, inplace=True)

# Generate pie chart
fig_5b = px.pie(group_5b, names='accept_coupon', title="'Bar' Coupon Acceptance Rate: Everyone other than 'drivers that go to bars more than once a month, without kid passengers, and had occupations other than farming, fishing, or forestry'")
fig_5b.show()

<b>Summary of coupon acceptance rates for groups in question 5:</b>
- Drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry: 
    - Acceptance Rate: 71.2%
- All Others: 
    - Acceptance Rate: 29.7%

#### 6. Compare the acceptance rates between those drivers who:

- go to bars more than once a month, had passengers that were not a kid, and were not widowed *OR*
- go to bars more than once a month and are under the age of 30 *OR*
- go to cheap restaurants more than 4 times a month and income is less than 50K. 



In [ ]:
# Explore values of "marital_status" column
bar_coupons['marital_status'].value_counts() 

In [ ]:
# Query 6a: "drivers go to bars more than once a month, had passengers that were not a kid, and were not widowed"
query_6a = "car != 'do not drive' & bar in @bc_more_than_once & passenger != 'Kid(s)' & marital_status != 'Widowed'"
group_6a = bar_coupons.query(query_6a)

# Generate pie chart
fig_6a = px.pie(group_6a, names='accept_coupon', title="'Bar' Coupon Acceptance Rate: Drivers that go to bars more than once a month, without kid passengers, and were not widowed")
fig_6a.show()


Note: the question asks for "drivers that go to bars more than once a month and are under the age of 30". However, since the age is given in age groups, people age 30 are included on the group "26 to 30". Therefore, I will provide both variants as part of my analysis below

In [ ]:
# Query 6b: "drivers that go to bars more than once a month and are under the age of 30 (please read note above)" 
age_under_30 = ['under 21', '21 to 25']
query_6b = "car != 'do not drive' & bar in @bc_more_than_once & age in @age_under_30"
group_6b = bar_coupons.query(query_6b)

# Generate pie chart
fig_6b = px.pie(group_6b, names='accept_coupon', title="'Bar' Coupon Acceptance Rate: Drivers that go to bars more than once a month and are under the age of 26")
fig_6b.show()

In [ ]:
# Query 6b: "drivers that go to bars more than once a month and are under the age of 31 (please read note above)" 
age_under_31 = ['under 21', '21 to 25', '26 to 30']
query_6b_variant = "car != 'do not drive' & bar in @bc_more_than_once and age in @age_under_31"
group_6b_variant = bar_coupons.query(query_6b_variant)

# Generate pie chart
fig_6b_variant = px.pie(group_6b_variant, names='accept_coupon', title="'Bar' Coupon Acceptance Rate: Drivers that go to bars more than once a month and are under the age of 31")
fig_6b_variant.show()

In [ ]:
# Set list of values for "more than four times a month"
more_than_four_tam = ['4~8', 'gt8']

# Explore income column values
bar_coupons['income'].value_counts()

In [ ]:
# Set list of values for "income less than 50k"
income_less_than_50k = ['Less than $12500', '$12500 - $24999', '$25000 - $37499', '$37500 - $49999']

# Query 6c: "drivers that go to cheap restaurants more than 4 times a month and income is less than 50K."
query_6c = "car != 'do not drive' & restaurant_less_than_20 in @more_than_four_tam & income in @income_less_than_50k"

# Set group 6c:
group_6c = bar_coupons.query(query_6c) 

# Generate visualization
fig_6c = px.pie(group_6c, names="accept_coupon", title="'Bar' Coupon Acceptance Rate: Drivers that go to cheap restaurants more than 4 times a month and income is less than 50K") 
fig_6c.show()


#### 7.  Based on these observations, what do you hypothesize about drivers who accepted the bar coupons?

##### Summary of the data collected:

- Overall acceptance rate of bar coupons: 41.0%
- Acceptance rate by group:
    - Those who went to a bar 3 or fewer times in a month: 37.1% 
    - Those who went to a bar more than 3 times in a month: 76.9%
    - Drivers who are over 25 years old and visit a bar more than once a month accepted the coupon: 69.4% vs 33.6% (all others)
    - Drivers who go to bars more than once a month and had passengers who were not a kid and had occupations other than farming, fishing, or forestry: 71.2% vs 29.7% (all others)
    - Drivers who go to bars more than once a month, had passengers who were not a kid, and were not widowed: 71.2%
    - Drivers who go to bars more than once a month and are under the age of 30: 67.0% 
    - Drivers who go to bars more than once a month and are under the age of 31: 72.0%
    - Drivers who go to cheap restaurants more than 4 times a month and income is less than 50K: 45.3% 


##### Hypothesis:
Based on the data obtained thus far, we can draw the following hypothesis:
The highest acceptance rate for bar coupons will come from those who go to a bar more than 3 times a month, followed by drivers aged under 31 years old who go to the bar more than once in a month. On the other hand, we may predict that the lowest acceptance rate for bar coupons will come from those who go to a bar 3 or less times in a month. 

-----------------------

## Independent Investigation

Using the bar coupon example as motivation, you are to explore one of the other coupon groups and try to determine the characteristics of passengers who accept the coupons.  

In [ ]:
# Query entries where accept_coupon == True
accepted_coupons_df = data.query("accept_coupon == True")

# Explore results
accepted_coupons_df.head()

In [ ]:
# Generate histogram to visualize distribution of coupon types 
fig = px.histogram(
    accepted_coupons_df,
    x='coupon' 
)
fig.show()

From the chart above we can see that the 'Coffee House', 'Restaurant(<20)' and 'Carry out & Take Away' coupon types are the most accepted.

In [ ]:
# Genarate historgram to visualize distribution of coupons by age and gender. Add "has_children" pattern
fig = px.histogram(
    accepted_coupons_df, 
    x='age', 
    color='gender', 
    category_orders={'age' : ["under 21", "21 to 25", "26 to 30", "31 to 35", "36 to 40", "41 to 45", "46 to 50", "Over 50"]}, 
    pattern_shape='has_children',
    title="Coupon Acceptance Count: Aggregated by age, gender and has children (true/false)"
    )
fig.show()

From the previous chart we can see that the the highest acceptance comes from the population of age "21 through 30", almost equally distributed among males and females, and with no children (as shown by the pattern). 

In [ ]:
# Filter data to only include only the population aged 21 to 30 years old
ac_21_to_30_df = accepted_coupons_df.query("age == '21 to 25' | age == '26 to 30'")

# Genarate histogram to visualize distribution by passenger and destination. Add "same_direction" as pattern
px.histogram(ac_21_to_30_df, x='passenger', color='destination', pattern_shape='direction_same', title="Coupon Acceptance Count: Aggregated by passenger type and destination")

From the chart above we can see that the highest acceptance comes from the population that drives alone, followed by the population that travel with friends. Some important observations from this chart:
1. There is a higher engagement from those traveling alone in opposite direction to work (as shown by the pattern). 
2. There is a higher engagement from those traveling alone in the same direction to home (as shown by the pattern). 
3. Those traveling with friends engage while traveling to a "No Urgent Place" (not going to work or home). 

In [ ]:
# Generate histogram to visualize distribution by income and occupation
px.histogram(
    ac_21_to_30_df, 
    x='income', 
    color='occupation',
    category_orders={
        'income' : [
            'Less than $12500',
            '$12500 - $24999',
            '$25000 - $37499',
            '$37500 - $49999',
            '$50000 - $62499',
            '$62500 - $74999',
            '$75000 - $87499',
            '$87500 - $99999',
            '$100000 or More'
        ]
    },
    title="Coupon Acceptance Count: Aggregated by income and occupation"
)

The chart above shows that the highest coupon acceptance comes from those with an income between $12,500 and $37,499. Those with occupation "Student" show higher coupon acceptance than others. 

To better visualize the coupon acceptance by occupation, we'll use a pie chart. 

In [ ]:
# Generate pie chart
px.pie(ac_21_to_30_df, names='occupation', title="Coupon Acceptance Rate: Aggregated by Occupation")

The previous chart shows that the highest coupon acceptance comes from those with occupations "Unemployed", "Student" and "Computer & Mathematical". The population with these three occupations represent slightly over half of the entire population that accepts coupons on the dataset. Therefore, we're going to focus on this population for the rest of the analysis.   

In [ ]:
# Create list of high engagement occupations 
filter_occupations = ['Unemployed', 'Student', 'Computer & Mathematical']

# Create dataframe with occupation filter 
ac_21_to_25_filtered_ccupations_df = ac_21_to_30_df.query("occupation in @filter_occupations")

# Generate pie chart to visualize car type
px.pie(ac_21_to_25_filtered_ccupations_df, names='car', title="Coupon Acceptance Rate: Aggregated by car type")

The pie chart above shows that 97.9% of the entries on this subset of the data contain null values on the "car" column. 

<i>Note: As discussed on the first part of this analysis, we are assuming that the person is still a driver (since these entries also contain information about the passenger), and that the null values represent missing information about the car type.</i> 

In [ ]:
# Generate pie chart to visualize marital status
px.pie(ac_21_to_25_filtered_ccupations_df, names='marital_status', title="Coupon Acceptance Rate: Aggregated by marital status")

The chart above shows that more than half of the population (57.9%) are single.

In [ ]:
# Generate histogram to visualize the distribution by weather and temperature 
px.histogram(ac_21_to_25_filtered_ccupations_df, x='weather', color="temperature", title="Coupon Acceptance Count: Aggregated by weather and temperature")

The chart above shows us that the vast majority of the coupons were accepted on a sunny day, with a higher proportion when the temperature is 80 F. 

In [ ]:
# Generate histogram to visualize distribution by time of acceptance and coupon expiration 
px.histogram(
    ac_21_to_25_filtered_ccupations_df, 
    x='time', 
    color='expiration',
    category_orders={
        'time' : [
            '7AM',
            '10AM',
            '2PM',
            '6PM',
            '10PM'
        ]
    },
    title="Coupon Acceptance Count: Aggregated by Time of Acceptance and Expiration"
)

From the previous visualization we can see that the highest acceptance is at 6PM, with a higher proportion on coupons expiring witin 1 day (1d).  

In [ ]:
# Generate histogram to visualize distribution by frequency of visits (per month) by establishement type  
px.histogram(
    ac_21_to_25_filtered_ccupations_df, 
    x=['coffee_house', 'bar', 'restaurant_less_than_20', 'restaurant_20_to_50', 'carry_away'],
    category_orders={
        'value' : [
            'never',
            'less1',
            '1~3',
            '4~8',
            'gt8'
        ]
    },
    title="Coupon Acceptance Count: Aggregated by frequency of visits (per month) and establishment type"
)

The visualization above shows us that the population with highest acceptance rate visits all establishment types 1~3 times per month, with a higher proportion on the "carry_away" establishement type.   

In [ ]:
# Generate piechart to visualize distribution by education level  
px.pie(ac_21_to_25_filtered_ccupations_df, names='education', title="Coupon Acceptance Rate: Aggregated by education level")

The pie chart above shows us that the highest coupon acceptance come at almost an exact proportion (~37%) from those with a "Bachelors degree" and those with "Some colle - no degree".  

In [ ]:
# Additional imports 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function: rename_columns
def rename_columns(x): 
    match x:
        case 'has_children':
            renamed_value = 'Has Children'
        case 'to_coupon_geq_15min':
            renamed_value = 'Distance To Coupon (>= 15min)'
        case 'to_coupon_geq_25min':
            renamed_value = 'Distance To Coupon (>= 25min)'
        case 'direction_same':
            renamed_value = 'Travels Same Direction'
        case 'direction_opp':
            renamed_value = 'Travels Opposite Direction'
    return renamed_value

# Create dataframe with subset of data
df = ac_21_to_25_filtered_ccupations_df[['has_children', 'to_coupon_geq_15min', 'to_coupon_geq_25min', 'direction_same', 'direction_opp']]

# Generate visualization
fig = make_subplots(rows=1, cols=5)
L= len(df)

# Set column names
cnames = list(df.columns)
for k, name in enumerate(cnames):
    n_true = df[name].sum()
    fig.add_trace(go.Bar(x=['True', 'False'], y=[n_true, L-n_true], name=rename_columns(name)), 1,k+1)

# Update layout
fig.update_layout(barmode='relative',  bargap=0.05, width=1600, height=400, title="Coupon Acceptance Count: Aggregated by common characteristics")



From the visualization above we see that the highest coupon acceptance come from the population with the following characteristics:
- Do not have children
- Distance to coupon within 15min and 25min
- Drive in the opposite direction to their destination

## Conclusion:

From the independent portion of this analysis, we draw the following conclusions about the characteristics of the population with the highest coupon engagement rate:

1. Their acceptance is higher for coupons of type "Coffee House", "Restaurant(<20)" and "Carry out & Take Away". 
2. They're of age "21 through 30", almost equally distributed among males and females, and with no children.   
3. Most of the population drives alone, followed by the population that travel with friends.
    - There is a higher engagement from those traveling alone in opposite direction to work. 
    - There is a higher engagement from those traveling alone in the same direction to home. 
    - Those traveling with friends engage while traveling to a "No Urgent Place" (not going to work or home).
4. They have an income between $12,500 and $37,499, and those with the occupation "Student" show higher acceptance than others. 
5. Their occupations are distributed among "Unemployed", "Student" and "Computer & Mathematical". The population with these three occupations represents slightly over half of the population that accepts coupons on the dataset. 
6. 97.9% of the entries on this subset of the data contain null values on the "car" column. For this analysis, we assumed that unless the car column contained the value "do not drive" the person is a driver. We support this assumption with the fact that the entries with NULL values also contain information about the passenger. Therefore, the assumption is that the null values represent missing information about the car type.
7. More than half of the population (57.9%) is single.
8. The vast majority of the coupons were accepted on a sunny day, with a higher proportion when the temperature is 80 F. 
9. The highest acceptance is at 6PM, with a higher proportion of coupons expiring within 1 day (1d).  
10. This population visits all establishment types 1~3 times per month, with a higher proportion on the "carry_away" establishment type.   
11. There's almost an exact proportion (~37%) of those with a "Bachelors degree" and those with "Some colle - no degree".  
12. Other characteristics about this population: 
	- They do not have children.
	- They accept the coupons when within a distance 15min and 25min to the establishment.
	- They drive in the opposite direction to their destination when they accept the coupon.


## Next Steps & Recommendations:
Although we've obtained valuable insights from this analysis, the fact that so many entries do not have a value (NULL) in the car column represents a concern about the results' quality. The assumption throughout this analysis was that entries without car information still meant drivers. This assumption is supported by most of those entries that also contained "passenger" information. Also, from the description provided with the dataset, it can be inferred that all entries had information about drivers.

However, a recommended next step would be to find out why this information was missing and try to obtain it from another source if possible. Once available, rerunning this analysis and comparing results would be best.